In [ ]:
#Pkg.checkout("DistributedArrays","anj/map_localparts")

In [26]:
1+3

4

In [ ]:
#using DistributedArrays

In [27]:
nprocs() == 1 && addprocs(4)
using DistributedArrays

In [28]:
@everywhere function prefix!(op, v, v0 = 0)
    v[1] += v0
    for i = 2:length(v)
        v[i] = op(v[i-1], v[i])
    end
    return v
end

@everywhere function prefixlag!(op, v, v0 = 0)
    vi = v[1]
    v[1] = v0
    for i = 2:length(v)
        tmp  = op(v[i - 1], vi)
        vi   = v[i]
        v[i] = tmp
    end
    v
end

In [29]:
prefixlag!(+,collect(1:8))

8-element Array{Int64,1}:
  0
  1
  3
  6
 10
 15
 21
 28

In [30]:
A = prefix!(*, [[0 1; 1 1] for i = 1:10])

10-element Array{Array{Int64,2},1}:
 [0 1; 1 1]    
 [1 1; 1 2]    
 [1 2; 2 3]    
 [2 3; 3 5]    
 [3 5; 5 8]    
 [5 8; 8 13]   
 [8 13; 13 21] 
 [13 21; 21 34]
 [21 34; 34 55]
 [34 55; 55 89]

In [31]:
prefix!(xor, bitrand(10))

10-element BitArray{1}:
  true
 false
 false
  true
 false
  true
 false
 false
 false
 false

In [32]:
x1, x2, x3, x4 = 0:9, 10:19, 20:29, 30:39

(0:9, 10:19, 20:29, 30:39)

In [38]:
x = Vector{Int}[x1, x2, x3, x4]

# Simulating local parts of 0:39 on four processors

4-element Array{Array{Int64,1},1}:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]          
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]

In [39]:
y1 = sum.(x)

4-element Array{Int64,1}:
  45
 145
 245
 345

In [40]:
y2 = prefixlag!(+, y1)

4-element Array{Int64,1}:
   0
  45
 190
 435

In [41]:
y3 = map((t,s) -> prefix!(+, t, s), x, y2)

4-element Array{Array{Int64,1},1}:
 [0, 1, 3, 6, 10, 15, 21, 28, 36, 45]              
 [55, 66, 78, 91, 105, 120, 136, 153, 171, 190]    
 [210, 231, 253, 276, 300, 325, 351, 378, 406, 435]
 [465, 496, 528, 561, 595, 630, 666, 703, 741, 780]

In [14]:
cumsum(0:39)'

1×40 RowVector{Int64,Array{Int64,1}}:
 0  1  3  6  10  15  21  28  36  45  55  …  561  595  630  666  703  741  780

## Distributed version (requires some changes to DArrays)
Notice similarity to and differences from the sequential version

In [42]:
xd = @DArray [i for i = 0:39]

40-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
  ⋮
 28
 29
 30
 31
 32
 33
 34
 35
 36
 37
 38
 39

In [43]:
[DistributedArrays.chunk(xd, i) for i = 1:4]

4-element Array{Array{Int64,1},1}:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]          
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]

In [44]:
yd1 = map_localparts(t -> [sum(t)], xd)

4-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
  45
 145
 245
 345

In [45]:
yd2 = prefixlag!(+, Array(yd1))

4-element Array{Int64,1}:
   0
  45
 190
 435

In [46]:
# Notice that we'll have to index into the offset of the prefix
# because `map_localpart` interprets each element as an Array
yd3 = map_localparts((t,s) -> prefix!(+, t, s[1]), xd, distribute(yd2))


40-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
   0
   1
   3
   6
  10
  15
  21
  28
  36
  45
  55
  66
  78
   ⋮
 406
 435
 465
 496
 528
 561
 595
 630
 666
 703
 741
 780

In [20]:
sum(vcat(x...) - yd3)

0

In [21]:
methods(map_localparts)

# 5 methods for generic function "map_localparts":
map_localparts(f::Union{Function, Type}, d::DistributedArrays.DArray) in DistributedArrays at /Users/alanedelman/.julia/v0.6/DistributedArrays/src/mapreduce.jl:185
map_localparts(f::Union{Function, Type}, d1::DistributedArrays.DArray, d2::DistributedArrays.DArray) in DistributedArrays at /Users/alanedelman/.julia/v0.6/DistributedArrays/src/mapreduce.jl:195
map_localparts(f::Union{Function, Type}, DA::DistributedArrays.DArray, A::Array) in DistributedArrays at /Users/alanedelman/.julia/v0.6/DistributedArrays/src/mapreduce.jl:205
map_localparts(f::Union{Function, Type}, A::Array, DA::DistributedArrays.DArray) in DistributedArrays at /Users/alanedelman/.julia/v0.6/DistributedArrays/src/mapreduce.jl:218
map_localparts(f::Union{Function, Type}, As::DistributedArrays.DArray...) in DistributedArrays at /Users/alanedelman/.julia/v0.6/DistributedArrays/src/mapreduce.jl:238

In [22]:
sin(cos(tan(5)))

-0.8257855101654086

In [23]:
tan(5) |> cos |> sin

-0.8257855101654086